# GPX FROM GARMIN
___

In [1]:
#%pip install --trusted-host files.pythonhosted.org --trusted-host pypi.org --trusted-host pypi.python.org lxml beautifulsoup4 numpy pandas

In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# FROM FILE TO DF
___

In [3]:
with open("gpx/СПУСК.gpx", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
soup = BeautifulSoup(text, "xml")

In [5]:
track = soup.find("trk")

In [6]:
track_points = track.find_all("trkpt")

In [7]:
lat_lst = []
lon_lst = []
ele_lst = []
time_lst = []

In [8]:
for i, point in enumerate(track_points):
    lat_lst.append(point.attrs["lat"])
    lon_lst.append(point.attrs["lon"])
    ele_lst.append(point.find("ele").text)
    time_lst.append(point.find("time").text)

In [9]:
df = pd.DataFrame({
    "lat": lat_lst,
    "lon": lon_lst,
    "ele": ele_lst,
    "datetime": time_lst
})

___
# PROCESSING
___

In [10]:
df["lon"] = df["lon"].astype(float)
df["lat"] = df["lat"].astype(float)
df["ele"] = df["ele"].astype(float)
df["datetime"] = pd.to_datetime(df["datetime"])

In [14]:
df["lat_shift"] = df["lat"].shift()
df["lon_shift"] = df["lon"].shift()

In [15]:
df

,lat,lon,ele,datetime,lat_shift,lon_shift
0,55.509563,38.346976,125.11,2025-07-07 16:57:30+00:00,NaN,NaN
1,55.509550,38.346937,125.59,2025-07-07 16:57:36+00:00,55.509563,38.346976
2,55.509575,38.346850,125.11,2025-07-07 16:57:38+00:00,55.509550,38.346937
3,55.509590,38.346827,125.11,2025-07-07 16:57:39+00:00,55.509575,38.346850
4,55.510107,38.346077,125.11,2025-07-07 16:57:55+00:00,55.509590,38.346827
5,55.510255,38.345885,125.11,2025-07-07 16:57:59+00:00,55.510107,38.346077
6,55.510292,38.345845,125.11,2025-07-07 16:58:00+00:00,55.510255,38.345885
7,55.510814,38.345078,124.15,2025-07-07 16:58:15+00:00,55.510292,38.345845
8,55.511246,38.344313,123.67,2025-07-07 16:58:32+00:00,55.510814,38.345078
9,55.511442,38.343974,123.67,2025-07-07 16:58:45+00:00,55.511246,38.344313


In [19]:
def dr(row):
    
    r = 6_371_000
    
    lat_1 = np.radians(row["lat"])
    lon_1 = np.radians(row["lon"])
    lat_2 = np.radians(row["lat_shift"])
    lon_2 = np.radians(row["lon_shift"])
    


    
    l = r * np.arccos(
        np.sin(lat_1) * np.sin(lat_2) + np.cos(lat_1) * np.cos(lat_2) * np.cos(lon_1 - lon_2)
    )
    
    return l

In [20]:
df["dr"] = df.apply(dr, axis=1)

In [21]:
df

,lat,lon,ele,datetime,lat_shift,lon_shift,dr
0,55.509563,38.346976,125.11,2025-07-07 16:57:30+00:00,NaN,NaN,NaN
1,55.509550,38.346937,125.59,2025-07-07 16:57:36+00:00,55.509563,38.346976,2.893581
2,55.509575,38.346850,125.11,2025-07-07 16:57:38+00:00,55.509550,38.346937,6.145915
3,55.509590,38.346827,125.11,2025-07-07 16:57:39+00:00,55.509575,38.346850,2.150237
4,55.510107,38.346077,125.11,2025-07-07 16:57:55+00:00,55.509590,38.346827,74.368825
5,55.510255,38.345885,125.11,2025-07-07 16:57:59+00:00,55.510107,38.346077,20.438227
6,55.510292,38.345845,125.11,2025-07-07 16:58:00+00:00,55.510255,38.345885,4.912844
7,55.510814,38.345078,124.15,2025-07-07 16:58:15+00:00,55.510292,38.345845,75.444120
8,55.511246,38.344313,123.67,2025-07-07 16:58:32+00:00,55.510814,38.345078,68.052784
9,55.511442,38.343974,123.67,2025-07-07 16:58:45+00:00,55.511246,38.344313,30.525057


In [22]:
df["dt"] = df["datetime"].diff().dt.total_seconds().fillna(0)

In [23]:
df["speed"] = round( ( df["dr"] / + df["dt"] ) * 3.6 , 2)

In [24]:
df = df.fillna(0)

In [25]:
df["ele_diff"] = df["ele"].diff()

___
# DATA
___

- DISTANCE BY GARMIN

In [26]:
distance = track.find("gpxtrkx:Distance").text
print ( "\t", distance )

	 287


- N POINTS

In [27]:
n_points = df["lat"].count()
print ( "\t", n_points )

	 11


- TRACK NAME

In [28]:
track_name = track.find("name").text
print ( "\t", track_name )

	 СПУСК


- MAX SPEED

In [29]:
max_speed = df["speed"].max()
print ( "\t", max_speed )

	 18.39


- AVG SPEED

In [30]:
avg_speed = round( df["dr"].sum() / df["dt"].sum() * 3.6, 2)
print ( "\t", avg_speed )

	 9.82


- DISTANCE

In [31]:
distance = round( df["dr"].sum(), 2)
print ( "\t", distance )

	 286.51


- LONGEST PART

In [32]:
longest_part = round( df["dr"].max(), 2)
print ( "\t", longest_part )

	 75.44


- SMALLEST PART

In [33]:
longest_part = round( df["dr"].min(), 2)
print ( "\t", longest_part )

	 0.0


- LONGEST TIME

In [34]:
longest_time = round( df["dt"].max(), 2)
print ( "\t", longest_time )

	 30.0


- SMALLEST TIME

In [35]:
smallest_time = round( df["dt"].min(), 2)
print ( "\t", smallest_time )

	 0.0


- TOTAL TIME

In [36]:
total_time = df["dt"].sum()
hours = total_time // 3600
minutes = ( total_time % 3600 ) // 60
seconds = int(( total_time % 3600 ) % 60)
print ("\t", int( total_time ), "s")
print ("\t", round( total_time / 60, 2 ), "m")
print ("\t", round( total_time / 3600, 2 ), "h")
print ("\t", hours, "h", minutes, "m", seconds, "s")

	 105 s
	 1.75 m
	 0.03 h
	 0.0 h 1.0 m 45 s


- MIN ELEVATION

In [37]:
min_ele = df["ele"].min()
print ( "\t", min_ele )

	 122.71


- MAX ELE

In [123]:
max_ele = df["ele"].max()
print ( "\t", max_ele )

	 125.59


- DOWN DISTANCE

In [124]:
down_distance = -round( df[ df["ele_diff"] < 0 ]["ele_diff"].sum(), 2 )
print ( "\t", down_distance )

	 2.88


- UP DISTANCE

In [125]:
up_distance = round( df[ df["ele_diff"] > 0 ]["ele_diff"].sum(), 2 )
print ( "\t", up_distance )

	 0.48


___

In [126]:
with open(f"{track_name}.txt", "w", encoding="UTF-8") as f:
    f.write(f"Поход {track_name}\n"
            f"Пройденное расстояние - {distance} м\n"
            f"Средняя скорость - {avg_speed} км/ч\n"
            f"Максимальная скорость - {max_speed} км/ч\n"
            f"Время похода - {hours} ч, {minutes} м, {seconds} с\n"
            f"Самая большая высота - {max_ele} м\n"
            f"Самая маленькая высота - {min_ele} м\n"
            f"Спуск - {down_distance} м\n"
            f"Подъем - {up_distance} м\n"
            )


In [89]:
df["coords"][0]

(np.float64(2838784.550004626),
 np.float64(2246046.6231034175),
 np.float64(5233946.618223295))

In [90]:
df["coords"][1]

(np.float64(2838785.7818863746),
 np.float64(2246046.3552860897),
 np.float64(5233944.321645297))

In [91]:
df

,lat,lon,ele,datetime,coords,dr,dt,speed,ele_diff
0,55.511593,38.351049,126.07,2025-07-07 16:55:39+00:00,"(2838784.550004626, 2246046.6231034175, 523394...",0.000000,0.0,0.00,NaN
1,55.511576,38.351033,124.63,2025-07-07 16:55:49+00:00,"(2838785.7818863746, 2246046.3552860897, 52339...",1.260658,10.0,0.45,-1.44
2,55.511498,38.350881,124.15,2025-07-07 16:55:54+00:00,"(2838797.0989851807, 2246043.080349284, 523393...",11.781423,5.0,8.48,-0.48
3,55.511466,38.350815,124.15,2025-07-07 16:55:55+00:00,"(2838802.0072532278, 2246041.6494307937, 52339...",5.112595,1.0,18.41,0.00
4,55.511447,38.350779,124.15,2025-07-07 16:55:56+00:00,"(2838804.7510274854, 2246040.8964052177, 52339...",2.845232,1.0,10.24,0.00
5,55.511083,38.350064,124.15,2025-07-07 16:56:09+00:00,"(2838859.0353345186, 2246026.1981024113, 52339...",56.239009,13.0,15.57,0.00
6,55.510495,38.348898,123.67,2025-07-07 16:56:28+00:00,"(2838946.830385684, 2246001.7288516024, 523387...",91.141183,19.0,17.27,-0.48
7,55.510009,38.347936,123.19,2025-07-07 16:56:45+00:00,"(2839019.263178489, 2245981.600184811, 5233844...",75.177608,17.0,15.92,-0.48
8,55.509615,38.347114,123.19,2025-07-07 16:57:02+00:00,"(2839079.8752401075, 2245963.287195691, 523381...",63.318146,17.0,13.41,0.00
9,55.509560,38.347023,124.15,2025-07-07 16:57:13+00:00,"(2839087.803399175, 2245962.212242962, 5233816...",8.000702,11.0,2.62,0.96
